In [1]:
import cv2
import numpy as np
import tensorflow as tf
import os
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D,Activation, BatchNormalization, Conv2DTranspose, Input, Lambda
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras 
import os.path
from utils_joscha import *

TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [2]:
print(tf.__version__)

NameError: name 'tf' is not defined

In [ ]:
path='data/rppg_faces/' # Cropped faces using Resnet-10 model
filenames=sorted_alphanumeric(os.listdir(path))
len(filenames)

In [ ]:
w=[];h=[]; window_size=(31,31)
for file in filenames:
    img=cv2.imread(path+file)
    if img.shape[1]>window_size[0]:
        if img.shape[0]>window_size[1]:
            w.append(img.shape[1])
            h.append(img.shape[0])
w=np.array(w)
h=np.array(h)

In [ ]:
t, arr = plt.subplots(1,2,figsize=(16,5))
arr[0].hist(np.array(w*h),bins=150);
arr[0].set_xlabel('Areas',fontsize=12)
arr[0].set_ylabel('Frequency',fontsize=12)

arr[1].hist(np.array(h/w),bins=150);
arr[1].set_xlabel('Aspect ratios',fontsize=12)
arr[1].set_ylabel('Frequency',fontsize=12)
#plt.savefig('areas_aspect_ratios_wdfe.pdf',dpi=300)

In [ ]:
x_train=[]; stride=30
for file in filenames:
    img=cv2.imread(path+file)
    img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img=convertRGB2CIELab(img)
    if img.shape[1]>window_size[0]:
        if img.shape[0]>window_size[1]:
            for i in range(int((img.shape[0]-window_size[1])/stride)):
                for j in range(int((img.shape[1]-window_size[0])/stride)):
                    x_train.append(img[i*stride:i*stride+window_size[1],j*stride:j*stride+window_size[0],:])
x_train=np.array(x_train)
x_train.shape

### Define the model

In [ ]:
def Elementwise_multiply(x):
    pixel_weights, image = x
    pixel_weights = tensorflow.keras.backend.expand_dims(pixel_weights, axis=-1) #could be K.stack([mask]*3, axis=-1) too 
    return pixel_weights*image

In [ ]:
inputs = Input(shape=(31,31,3))
x = Conv2D(filters=16,kernel_size=7)(inputs)
x = BatchNormalization(axis=-1)(x)
x = Activation('relu')(x)
x = Conv2D(filters=32,kernel_size=7)(x)
x = BatchNormalization(axis=-1)(x)
x = Activation('relu')(x)
x = Conv2D(filters=32,kernel_size=7)(x)
x = BatchNormalization(axis=-1)(x)
x = Activation('relu')(x)
x = Conv2D(filters=64,kernel_size=7)(x)
x = BatchNormalization(axis=-1)(x)
x = Activation('relu')(x)
x = Conv2D(filters=128,kernel_size=7)(x)
x = BatchNormalization(axis=-1)(x)
x = Activation('relu')(x)
x = Conv2DTranspose(filters=64,kernel_size=7)(x)
x = BatchNormalization(axis=-1)(x)
x = Activation('relu')(x)
x = Conv2DTranspose(filters=32,kernel_size=7)(x)
x = BatchNormalization(axis=-1)(x)
x = Activation('relu')(x)
x = Conv2DTranspose(filters=32,kernel_size=7)(x)
x = BatchNormalization(axis=-1)(x)
x = Activation('relu')(x)
x = Conv2DTranspose(filters=16,kernel_size=7)(x)
x = BatchNormalization(axis=-1)(x)
x = Activation('relu')(x)
x = Conv2DTranspose(filters=3,kernel_size=7)(x)
x = BatchNormalization(axis=-1)(x)
x = Activation('sigmoid',name='sigmoid_output')(x)
outputs = Lambda(Elementwise_multiply)([get_gaussian_filter((31,31),5),x])
model = Model(inputs=inputs, outputs=outputs)

model=Sequential([Conv2D(filters=16,kernel_size=7,input_shape=(31,31,3)),
                  BatchNormalization(axis=-1),
                  Activation('relu'),
                  Conv2D(filters=32,kernel_size=7),
                  BatchNormalization(axis=-1),
                  Activation('relu'),
                  Conv2D(filters=32,kernel_size=7),
                  BatchNormalization(axis=-1),
                  Activation('relu'),
                  Conv2D(filters=64,kernel_size=7),
                  BatchNormalization(axis=-1),
                  Activation('relu'),
                  Conv2D(filters=128,kernel_size=7),
                  BatchNormalization(axis=-1),
                  Activation('relu'),
                  Conv2DTranspose(filters=64,kernel_size=7),
                  BatchNormalization(axis=-1),
                  Activation('relu'),
                  Conv2DTranspose(filters=32,kernel_size=7),
                  BatchNormalization(axis=-1),
                  Activation('relu'),
                  Conv2DTranspose(filters=32,kernel_size=7),
                  BatchNormalization(axis=-1),
                  Activation('relu'),
                  Conv2DTranspose(filters=16,kernel_size=7),
                  BatchNormalization(axis=-1),
                  Activation('relu'),
                  Conv2DTranspose(filters=3,kernel_size=7),
                  BatchNormalization(axis=-1),
                  Activation('sigmoid'),
                 ])

In [ ]:
def weighted_mse(y_true, y_pred):
    squared_difference = tf.reduce_mean(tf.square(y_true - y_pred),axis=2)
    pixel_weights = tf.cast(get_gaussian_filter((31,31),5),dtype=tf.float32)
    pixel_weights = tf.stack([pixel_weights]*32)
    return tf.multiply(squared_difference, pixel_weights)

In [ ]:
model.compile(optimizer='adamax',loss='mse')

In [ ]:
#model.summary()

In [ ]:
class CustomModelCheckPoint(tensorflow.keras.callbacks.Callback):
    def __init__(self,**kargs):
        super(CustomModelCheckPoint,self).__init__(**kargs)
        self.epoch_accuracy = {} # loss at given epoch
        self.epoch_loss = {} # accuracy at given epoch

    def on_epoch_begin(self,epoch, logs={}):
        # Things done on beginning of epoch. 
        return

    def on_epoch_end(self, epoch, logs={}):
        # things done on end of the epoch
        if epoch%50==0:
            path_losses = 'losses/dfe_wmse_'+str(epoch)+'.npy'
 
            self.epoch_loss[epoch] = logs.get("loss")
            np.save('losses/dfe_wmse_'+str(epoch),list(self.epoch_loss.values()))
            print(self.epoch_loss)
            self.model.save('models/wDFE_'+str(epoch)+'epochs.h5') # save the model


In [ ]:
x_train_weighted = np.zeros_like(x_train)
pixel_weights = np.swapaxes([get_gaussian_filter((31,31),5)]*3,0,2)
for i,x in enumerate(x_train):
    x_train_weighted[i] = x*pixel_weights

In [ ]:
checkpoint = CustomModelCheckPoint()
datagen = ImageDataGenerator()
histogram = model.fit_generator(datagen.flow(x_train, x_train_weighted, batch_size=32),
                    steps_per_epoch=len(x_train) / 32, epochs=1000, callbacks=[checkpoint])

In [ ]:
reduced_model= Model(inputs=model.input, outputs=model.get_layer('sigmoid_output').output)

In [ ]:
model.summary()

In [ ]:
reduced_model.summary()

In [ ]:
preds=reduced_model.predict(x_train)
preds.shape

In [ ]:
idx=10000
t, arr = plt.subplots(1,2,figsize=(16,5))
arr[0].imshow(convertCIELAB2RGB(preds[idx]))
arr[1].imshow(convertCIELAB2RGB(x_train[idx]))

In [ ]:
np.load('losses/dfe_wmse_500.npy')

In [ ]:
wdfe = load_model('models/DFE_500.h5')